In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from collections import defaultdict
import tabulate
import csv
import pickle
from numpy import genfromtxt
from recsysNN_utils import *
import numpy as np 
from tensorflow import keras
from matplotlib import pyplot as plt

In [2]:
x_item = pd.read_csv("csv/x_item.csv")

In [3]:
movie_dict = defaultdict(dict)
count = 0
with open('csv/movie_list.csv', newline='',encoding="utf8") as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count += 1  #skip header
                #print(line) print
            else:
                count += 1
                movie_id = int(line[0])
                movie_dict[movie_id]["title"] = line[1]
                movie_dict[movie_id]["genres"] = line[2]

In [4]:
print(f'train_item shape:{x_item.shape}')

train_item shape:(5630130, 17)


In [5]:
# scale training data
unscaled_train_item = x_item

scalerItem = StandardScaler()#scalerItem = StandardScaler()
scalerItem.fit(x_item)#scalerItem.fit(item_train)
x_item = scalerItem.transform(x_item)#item_train = scalerItem.transform(item_train)

'scalerUser = StandardScaler()#scalerUser = StandardScaler()\nscalerUser.fit(x_user)#scalerUser.fit(user_train)\nx_user = scalerUser.transform(x_user)#user_train = scalerUser.transform(user_train)\n\nscalerTarget = MinMaxScaler((-1, 1))#scalerTarget = MinMaxScaler((-1, 1))\nscalerTarget.fit(y.values.reshape(-1, 1))#scalerTarget.fit(y_train.reshape(-1, 1))\ny_train = scalerTarget.transform(y.values.reshape(-1, 1))#y_train = scalerTarget.transform(y_train.reshape(-1, 1))\n#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))\n\nprint(np.allclose(unscaled_train_item, scalerItem.inverse_transform(x_item)))\nprint(np.allclose(unscaled_train_user, scalerUser.inverse_transform(x_user)))'

In [6]:
num_user_features = 17 - 3
num_item_features = 17 - 1
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items

In [7]:
num_outputs = 32
input_item_m = tf.keras.layers.Input(shape=(num_item_features))    # input layer
item_NN = tf.keras.models.Sequential([
      
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs),
    
])
vm_m = item_NN(input_item_m)                                       # use the trained item_NN
vm_m = tf.linalg.l2_normalize(vm_m, axis=1)                        # incorporate normalization as was done in the original model
model_m = tf.keras.Model(input_item_m, vm_m)                                
model_m.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 16)]              0         
                                                                 
 sequential (Sequential)     (None, 32)                57952     
                                                                 
 tf.math.l2_normalize (TFOpL  (None, 32)               0         
 ambda)                                                          
                                                                 
Total params: 57,952
Trainable params: 57,952
Non-trainable params: 0
_________________________________________________________________


In [8]:
item_vecs = genfromtxt('csv/item_vecs.csv', delimiter=',')

In [9]:
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:,i_s:])
print(f"size of all predicted movie feature vectors: {vms.shape}")

c:\Users\Burak\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


27/27 [==============================] - 0s 1ms/step
size of all predicted movie feature vectors: (860, 32)


In [10]:
def sq_dist(a,b):
    """
    Returns the squared distance between two vectors
    Args:
      a (ndarray (n,)): vector with n features
      b (ndarray (n,)): vector with n features
    Returns:
      d (float) : distance
    """
    n = len(a)
    temp = []
    d = 0 
    temp.append(np.square([a-b]))
    d = np.sum(temp)
    
      
    return d

In [11]:
count = 50  # number of movies to display
dim = len(vms)
dist = np.zeros((dim,dim))

for i in range(dim):
    for j in range(dim):
        dist[i,j] = sq_dist(vms[i, :], vms[j, :])
        
m_dist = ma.masked_array(dist, mask=np.identity(dist.shape[0]))  # mask the diagonal


'disp = [["movie1", "genres", "movie2", "genres"]]\nfor i in range(count):\n    min_idx = np.argmin(m_dist[i])\n    movie1_id = int(item_vecs[i,0])\n    movie2_id = int(item_vecs[min_idx,0])\n    disp.append( [movie_dict[movie1_id][\'title\'], movie_dict[movie1_id][\'genres\'],\n                  movie_dict[movie2_id][\'title\'], movie_dict[movie1_id][\'genres\']]\n               )\ntable = tabulate.tabulate(disp, tablefmt=\'html\', headers="firstrow")\ntable'

In [17]:
with open('./pickles/m_dist.pickle', 'wb') as f:
    # Dump the dictionary into the file using pickle
    pickle.dump(m_dist, f)

In [18]:
with open('./pickles/movie_dict.pickle', 'wb') as f:
    # Dump the dictionary into the file using pickle
    pickle.dump(movie_dict, f)

In [14]:
np.argsort(m_dist[10])[::1][:10]

array([138, 348, 161, 544,  28,  35, 621,  89, 458, 154], dtype=int64)

In [15]:
movie_dict[item_vecs[676,0]]['title']

'Troy (2004)'

In [16]:
for i in np.argsort(m_dist[676])[::1][:10]:
    print(movie_dict[item_vecs[i,0]]['title'])

Hero (Ying xiong) (2002)
Gladiator (2000)
Last Samurai, The (2003)
Into the Wild (2007)
King Kong (2005)
Thor (2011)
Harry Potter and the Order of the Phoenix (2007)
Apollo 13 (1995)
The Count of Monte Cristo (2002)
Dances with Wolves (1990)
